In [0]:
!pip install torch torchvision tqdm
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5968e000 @  0x7fb3ed1f31c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 51kB 11.7MB/s 
    100% |████████████████████████████████| 2.0MB 2.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [0]:
def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))

In [0]:
def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
                                    100 * correct / total))

In [0]:
def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))


In [31]:
num_epochs = 5
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=5)#32-5+1=28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#14
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)#14-3+1=12
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#6
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)#6-3+1=4
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#2
        self.conv4 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1)#2
        self.pool4= nn.MaxPool2d(kernel_size=2, stride=2)#1
        self.fc1 = nn.Linear(in_features=1024*1*1, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        #self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        
        x = x.view(-1, 1024*1*1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')




  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1





  0%|          | 2/12500 [00:00<11:33, 18.01it/s]


  0%|          | 9/12500 [00:00<05:02, 41.36it/s]


  0%|          | 16/12500 [00:00<04:09, 50.00it/s]


  0%|          | 23/12500 [00:00<03:51, 54.01it/s]


  0%|          | 30/12500 [00:00<03:40, 56.51it/s]


  0%|          | 36/12500 [00:00<03:42, 56.05it/s]


  0%|          | 43/12500 [00:00<03:38, 57.04it/s]


  0%|          | 50/12500 [00:00<03:33, 58.23it/s]


  0%|          | 57/12500 [00:00<03:29, 59.27it/s]


  1%|          | 64/12500 [00:01<03:26, 60.08it/s]


  1%|          | 71/12500 [00:01<03:24, 60.87it/s]


  1%|          | 78/12500 [00:01<03:23, 60.95it/s]


  1%|          | 85/12500 [00:01<03:22, 61.42it/s]


  1%|          | 92/12500 [00:01<03:21, 61.60it/s]


  1%|          | 99/12500 [00:01<03:19, 62.01it/s]


  1%|          | 106/12500 [00:01<03:20, 61.92it/s]


  1%|          | 113/12500 [00:01<03:23, 60.86it/s]


  1%|          | 119/12500 [00:02<03:28, 59.35it/s]


  1%|          | 125/12500 [00:02<03:31, 5

epoch 1 training loss: 1.948





  0%|          | 1/2500 [00:00<06:12,  6.70it/s]


  0%|          | 12/2500 [00:00<00:52, 47.35it/s]


  1%|          | 29/2500 [00:00<00:30, 80.25it/s]


  2%|▏         | 38/2500 [00:00<00:30, 81.75it/s]


  2%|▏         | 46/2500 [00:00<00:30, 80.08it/s]


  2%|▏         | 55/2500 [00:00<00:30, 80.63it/s]


  3%|▎         | 65/2500 [00:00<00:29, 82.28it/s]


  3%|▎         | 74/2500 [00:00<00:30, 80.79it/s]


  3%|▎         | 84/2500 [00:01<00:29, 82.09it/s]


  4%|▎         | 93/2500 [00:01<00:29, 82.37it/s]


  4%|▍         | 103/2500 [00:01<00:28, 83.88it/s]


  5%|▍         | 113/2500 [00:01<00:28, 84.73it/s]


  5%|▌         | 126/2500 [00:01<00:27, 87.87it/s]


  6%|▌         | 144/2500 [00:01<00:25, 93.62it/s]


  6%|▋         | 159/2500 [00:01<00:24, 96.83it/s]


  7%|▋         | 172/2500 [00:01<00:24, 95.33it/s]


  7%|▋         | 184/2500 [00:01<00:24, 94.01it/s]


  8%|▊         | 195/2500 [00:02<00:24, 94.10it/s]


  8%|▊         | 206/2500 [00:02<00:24, 92.86it/s]


 

...

epoch 5 training loss: 0.804





  0%|          | 1/2500 [00:00<07:05,  5.87it/s]


  0%|          | 8/2500 [00:00<01:22, 30.04it/s]


  1%|          | 17/2500 [00:00<00:54, 45.49it/s]


  1%|          | 27/2500 [00:00<00:44, 56.09it/s]


  2%|▏         | 43/2500 [00:00<00:33, 73.62it/s]


  2%|▏         | 59/2500 [00:00<00:28, 86.20it/s]


  3%|▎         | 77/2500 [00:00<00:24, 97.60it/s]


  4%|▍         | 94/2500 [00:00<00:22, 105.40it/s]


  4%|▍         | 108/2500 [00:01<00:22, 106.13it/s]


  5%|▍         | 122/2500 [00:01<00:23, 100.74it/s]


  5%|▌         | 134/2500 [00:01<00:23, 99.09it/s] 


  6%|▌         | 151/2500 [00:01<00:22, 103.76it/s]


  7%|▋         | 168/2500 [00:01<00:21, 107.68it/s]


  7%|▋         | 185/2500 [00:01<00:20, 111.35it/s]


  8%|▊         | 203/2500 [00:01<00:19, 115.00it/s]


  9%|▉         | 219/2500 [00:01<00:19, 114.46it/s]


  9%|▉         | 233/2500 [00:02<00:20, 112.35it/s]


 10%|▉         | 246/2500 [00:02<00:20, 111.47it/s]


 10%|█         | 260/2500 [00:02<00:19, 11

Accuracy of the network on the 10000 test images: 67 %





  0%|          | 1/2500 [00:00<04:27,  9.36it/s]


  1%|          | 15/2500 [00:00<00:34, 71.56it/s]


  1%|          | 30/2500 [00:00<00:25, 96.01it/s]


  2%|▏         | 45/2500 [00:00<00:22, 107.79it/s]


  2%|▏         | 60/2500 [00:00<00:21, 114.49it/s]


  3%|▎         | 73/2500 [00:00<00:20, 116.53it/s]


  4%|▎         | 88/2500 [00:00<00:20, 120.56it/s]


  4%|▍         | 103/2500 [00:00<00:19, 123.49it/s]


  5%|▍         | 117/2500 [00:00<00:19, 124.93it/s]


  5%|▌         | 132/2500 [00:01<00:18, 126.71it/s]


  6%|▌         | 147/2500 [00:01<00:18, 128.13it/s]


  6%|▋         | 161/2500 [00:01<00:18, 128.71it/s]


  7%|▋         | 176/2500 [00:01<00:17, 129.79it/s]


  8%|▊         | 192/2500 [00:01<00:17, 131.23it/s]


  8%|▊         | 207/2500 [00:01<00:17, 132.22it/s]


  9%|▉         | 222/2500 [00:01<00:17, 133.01it/s]


  9%|▉         | 237/2500 [00:01<00:17, 132.86it/s]


 10%|█         | 252/2500 [00:01<00:16, 133.48it/s]


 11%|█         | 267/2500 [00:01<00:

Accuracy of plane : 73 %
Accuracy of   car : 87 %
Accuracy of  bird : 68 %
Accuracy of   cat : 53 %
Accuracy of  deer : 46 %
Accuracy of   dog : 73 %
Accuracy of  frog : 69 %
Accuracy of horse : 52 %
Accuracy of  ship : 83 %
Accuracy of truck : 65 %
Finished Training


In [32]:
print('Training the model: 6th to 10th epoch')
for epoch in range(6,10):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')




  0%|          | 0/12500 [00:00<?, ?it/s]

Training the model: 6th to 10th epoch
epoch  7





  0%|          | 1/12500 [00:00<21:44,  9.58it/s]


  0%|          | 9/12500 [00:00<05:05, 40.92it/s]


  0%|          | 15/12500 [00:00<04:27, 46.59it/s]


  0%|          | 22/12500 [00:00<04:01, 51.62it/s]


  0%|          | 30/12500 [00:00<03:42, 56.09it/s]


  0%|          | 38/12500 [00:00<03:31, 59.00it/s]


  0%|          | 46/12500 [00:00<03:22, 61.41it/s]


  0%|          | 54/12500 [00:00<03:17, 62.93it/s]


  0%|          | 62/12500 [00:00<03:13, 64.17it/s]


  1%|          | 70/12500 [00:01<03:11, 65.07it/s]


  1%|          | 78/12500 [00:01<03:08, 65.92it/s]


  1%|          | 86/12500 [00:01<03:06, 66.55it/s]


  1%|          | 94/12500 [00:01<03:05, 67.00it/s]


  1%|          | 102/12500 [00:01<03:05, 66.86it/s]


  1%|          | 110/12500 [00:01<03:03, 67.35it/s]


  1%|          | 118/12500 [00:01<03:03, 67.61it/s]


  1%|          | 126/12500 [00:01<03:01, 68.00it/s]


  1%|          | 134/12500 [00:01<03:01, 68.19it/s]


  1%|          | 142/12500 [00:02<03:00,

epoch 7 training loss: 0.661





  0%|          | 1/2500 [00:00<05:35,  7.46it/s]


  0%|          | 11/2500 [00:00<00:52, 47.30it/s]


  1%|          | 19/2500 [00:00<00:43, 56.53it/s]


  1%|          | 28/2500 [00:00<00:38, 63.58it/s]


  2%|▏         | 39/2500 [00:00<00:34, 71.75it/s]


  2%|▏         | 48/2500 [00:00<00:33, 73.77it/s]


  3%|▎         | 63/2500 [00:00<00:29, 83.42it/s]


  3%|▎         | 80/2500 [00:00<00:25, 93.55it/s]


  4%|▍         | 97/2500 [00:00<00:23, 101.34it/s]


  5%|▍         | 114/2500 [00:01<00:22, 107.33it/s]


  5%|▌         | 129/2500 [00:01<00:22, 105.83it/s]


  6%|▌         | 142/2500 [00:01<00:22, 104.28it/s]


  6%|▌         | 154/2500 [00:01<00:22, 102.56it/s]


  7%|▋         | 165/2500 [00:01<00:22, 101.83it/s]


  7%|▋         | 176/2500 [00:01<00:22, 101.93it/s]


  7%|▋         | 187/2500 [00:01<00:22, 102.13it/s]


  8%|▊         | 198/2500 [00:01<00:22, 102.10it/s]


  8%|▊         | 209/2500 [00:02<00:22, 102.33it/s]


  9%|▉         | 220/2500 [00:02<00:22, 102

...

epoch 10 training loss: 0.329





  0%|          | 1/2500 [00:00<04:36,  9.05it/s]


  1%|          | 16/2500 [00:00<00:32, 75.98it/s]


  1%|▏         | 32/2500 [00:00<00:23, 103.01it/s]


  2%|▏         | 48/2500 [00:00<00:21, 115.74it/s]


  3%|▎         | 64/2500 [00:00<00:19, 123.83it/s]


  3%|▎         | 80/2500 [00:00<00:18, 129.49it/s]


  4%|▍         | 96/2500 [00:00<00:18, 132.72it/s]


  4%|▍         | 112/2500 [00:00<00:17, 135.61it/s]


  5%|▌         | 128/2500 [00:00<00:17, 137.78it/s]


  6%|▌         | 144/2500 [00:01<00:16, 139.45it/s]


  6%|▋         | 160/2500 [00:01<00:16, 139.73it/s]


  7%|▋         | 175/2500 [00:01<00:16, 140.15it/s]


  8%|▊         | 190/2500 [00:01<00:16, 140.37it/s]


  8%|▊         | 205/2500 [00:01<00:16, 140.13it/s]


  9%|▉         | 221/2500 [00:01<00:16, 141.12it/s]


  9%|▉         | 236/2500 [00:01<00:15, 141.59it/s]


 10%|█         | 252/2500 [00:01<00:15, 142.33it/s]


 11%|█         | 267/2500 [00:01<00:15, 142.13it/s]


 11%|█▏        | 283/2500 [00:01<00

Accuracy of the network on the 10000 test images: 74 %





  0%|          | 1/2500 [00:00<04:27,  9.34it/s]


  1%|          | 13/2500 [00:00<00:40, 61.50it/s]


  1%|          | 27/2500 [00:00<00:28, 85.38it/s]


  2%|▏         | 41/2500 [00:00<00:24, 98.36it/s]


  2%|▏         | 53/2500 [00:00<00:24, 101.33it/s]


  3%|▎         | 67/2500 [00:00<00:22, 107.10it/s]


  3%|▎         | 80/2500 [00:00<00:21, 110.12it/s]


  4%|▍         | 94/2500 [00:00<00:21, 113.32it/s]


  4%|▍         | 108/2500 [00:00<00:20, 115.77it/s]


  5%|▍         | 122/2500 [00:01<00:20, 117.54it/s]


  5%|▌         | 136/2500 [00:01<00:19, 119.14it/s]


  6%|▌         | 150/2500 [00:01<00:19, 120.52it/s]


  7%|▋         | 164/2500 [00:01<00:19, 121.18it/s]


  7%|▋         | 178/2500 [00:01<00:19, 121.96it/s]


  8%|▊         | 192/2500 [00:01<00:18, 122.43it/s]


  8%|▊         | 206/2500 [00:01<00:18, 123.02it/s]


  9%|▉         | 220/2500 [00:01<00:18, 122.27it/s]


  9%|▉         | 233/2500 [00:01<00:18, 122.59it/s]


 10%|▉         | 246/2500 [00:02<00:18

Accuracy of plane : 80 %
Accuracy of   car : 88 %
Accuracy of  bird : 63 %
Accuracy of   cat : 62 %
Accuracy of  deer : 74 %
Accuracy of   dog : 52 %
Accuracy of  frog : 77 %
Accuracy of horse : 86 %
Accuracy of  ship : 81 %
Accuracy of truck : 76 %
Finished Training


In [33]:
print('Training the model: 11th to 15th epoch')
for epoch in range(10,15):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')




  0%|          | 0/12500 [00:00<?, ?it/s]

Training the model: 11th to 15th epoch
epoch  11





  0%|          | 1/12500 [00:00<22:17,  9.34it/s]


  0%|          | 7/12500 [00:00<06:40, 31.23it/s]


  0%|          | 14/12500 [00:00<04:54, 42.40it/s]


  0%|          | 22/12500 [00:00<04:10, 49.87it/s]


  0%|          | 29/12500 [00:00<03:55, 53.04it/s]


  0%|          | 36/12500 [00:00<03:45, 55.33it/s]


  0%|          | 43/12500 [00:00<03:41, 56.28it/s]


  0%|          | 49/12500 [00:00<03:41, 56.09it/s]


  0%|          | 56/12500 [00:00<03:37, 57.26it/s]


  1%|          | 63/12500 [00:01<03:34, 58.05it/s]


  1%|          | 70/12500 [00:01<03:31, 58.80it/s]


  1%|          | 77/12500 [00:01<03:29, 59.41it/s]


  1%|          | 84/12500 [00:01<03:27, 59.76it/s]


  1%|          | 91/12500 [00:01<03:28, 59.48it/s]


  1%|          | 99/12500 [00:01<03:25, 60.39it/s]


  1%|          | 106/12500 [00:01<03:24, 60.55it/s]


  1%|          | 113/12500 [00:01<03:25, 60.38it/s]


  1%|          | 120/12500 [00:01<03:25, 60.10it/s]


  1%|          | 127/12500 [00:02<03:26, 5

epoch 11 training loss: 0.267





  0%|          | 1/2500 [00:00<04:43,  8.80it/s]


  1%|          | 17/2500 [00:00<00:31, 78.44it/s]


  1%|▏         | 34/2500 [00:00<00:23, 106.94it/s]


  2%|▏         | 52/2500 [00:00<00:19, 122.87it/s]


  3%|▎         | 68/2500 [00:00<00:18, 128.56it/s]


  3%|▎         | 81/2500 [00:00<00:19, 121.57it/s]


  4%|▎         | 93/2500 [00:00<00:20, 116.91it/s]


  4%|▍         | 104/2500 [00:00<00:21, 112.49it/s]


  5%|▍         | 115/2500 [00:01<00:21, 109.13it/s]


  5%|▌         | 129/2500 [00:01<00:21, 111.75it/s]


  6%|▌         | 141/2500 [00:01<00:21, 112.21it/s]


  6%|▋         | 158/2500 [00:01<00:20, 116.22it/s]


  7%|▋         | 175/2500 [00:01<00:19, 119.44it/s]


  8%|▊         | 189/2500 [00:01<00:19, 119.63it/s]


  8%|▊         | 203/2500 [00:01<00:19, 116.31it/s]


  9%|▊         | 215/2500 [00:01<00:20, 113.64it/s]


  9%|▉         | 226/2500 [00:02<00:20, 111.56it/s]


 10%|▉         | 242/2500 [00:02<00:19, 113.71it/s]


 10%|█         | 259/2500 [00:02<00

...

epoch 15 training loss: 0.124





  0%|          | 3/2500 [00:00<01:32, 27.09it/s]


  0%|          | 9/2500 [00:00<00:58, 42.88it/s]


  1%|          | 17/2500 [00:00<00:46, 53.26it/s]


  1%|          | 26/2500 [00:00<00:40, 60.92it/s]


  1%|▏         | 34/2500 [00:00<00:38, 64.60it/s]


  2%|▏         | 44/2500 [00:00<00:35, 70.08it/s]


  2%|▏         | 55/2500 [00:00<00:32, 74.72it/s]


  3%|▎         | 66/2500 [00:00<00:31, 78.32it/s]


  3%|▎         | 76/2500 [00:00<00:30, 80.49it/s]


  3%|▎         | 87/2500 [00:01<00:29, 83.10it/s]


  4%|▍         | 98/2500 [00:01<00:28, 84.96it/s]


  4%|▍         | 109/2500 [00:01<00:27, 86.76it/s]


  5%|▌         | 126/2500 [00:01<00:25, 92.67it/s]


  6%|▌         | 138/2500 [00:01<00:25, 93.07it/s]


  6%|▌         | 150/2500 [00:01<00:25, 92.34it/s]


  6%|▋         | 161/2500 [00:01<00:25, 91.49it/s]


  7%|▋         | 171/2500 [00:01<00:25, 91.25it/s]


  7%|▋         | 181/2500 [00:01<00:25, 91.51it/s]


  8%|▊         | 191/2500 [00:02<00:25, 91.45it/s]


  8

Accuracy of the network on the 10000 test images: 73 %





  0%|          | 1/2500 [00:00<10:01,  4.16it/s]


  0%|          | 9/2500 [00:00<01:33, 26.56it/s]


  1%|          | 18/2500 [00:00<01:01, 40.33it/s]


  1%|          | 27/2500 [00:00<00:50, 48.67it/s]


  1%|▏         | 36/2500 [00:00<00:45, 54.28it/s]


  2%|▏         | 45/2500 [00:00<00:42, 58.36it/s]


  2%|▏         | 54/2500 [00:00<00:39, 62.03it/s]


  3%|▎         | 67/2500 [00:00<00:35, 68.91it/s]


  3%|▎         | 82/2500 [00:01<00:31, 76.17it/s]


  4%|▎         | 93/2500 [00:01<00:30, 78.17it/s]


  4%|▍         | 104/2500 [00:01<00:31, 77.29it/s]


  5%|▍         | 114/2500 [00:01<00:31, 75.72it/s]


  5%|▍         | 123/2500 [00:01<00:31, 76.14it/s]


  5%|▌         | 132/2500 [00:01<00:30, 76.59it/s]


  6%|▌         | 141/2500 [00:01<00:30, 76.86it/s]


  6%|▌         | 150/2500 [00:01<00:30, 77.51it/s]


  6%|▋         | 159/2500 [00:02<00:30, 77.70it/s]


  7%|▋         | 168/2500 [00:02<00:29, 78.20it/s]


  7%|▋         | 181/2500 [00:02<00:28, 80.48it/s]


  

Accuracy of plane : 72 %
Accuracy of   car : 85 %
Accuracy of  bird : 75 %
Accuracy of   cat : 50 %
Accuracy of  deer : 56 %
Accuracy of   dog : 64 %
Accuracy of  frog : 85 %
Accuracy of horse : 72 %
Accuracy of  ship : 85 %
Accuracy of truck : 84 %
Finished Training
